In [7]:
import sys
from pathlib import Path
import numpy as np
import polars as pl
import pickle

current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

from src import (
    DataConfig, DataLoader, ModelingStrategy, ReleaseManager, BenchmarkPipeline, create_config
)

np.random.seed(42)
print(current_dir)

/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking


In [25]:
data_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
df_clean = pl.read_ipc(data_path)

sku_tuples_complete = [(d['productID'], d['storeID']) for d in df_clean.select(pl.col("productID"), pl.col("storeID")).unique().to_dicts()]
print("total unseen skus: ", len(sku_tuples_complete))

total unseen skus:  7634


In [ ]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather",
    target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather",
    split_date="2016-01-01",
)

sku_tuples=[(1912, 7), (377, 1), (715, 7)]

quantiles = [0.5, 0.7, 0.9, 0.95, 0.99]
pipeline = BenchmarkPipeline(data_config)

total unseen skus:  7634


In [9]:
results_lightning_std = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="xgboost_quantile",
    quantile_alphas=quantiles,
    hyperparameters = {
        "eta": 0.05,
        "max_depth": 8,
        "min_child_weight": 20,
        "subsample": 0.8,
        "colsample_bytree": 0.7,
        "gamma": 1.0,   
        "lambda": 10.0,
        "alpha": 1.0,
        "tree_method": "hist",
        "num_boost_round": 100,
        "seed": 42
},
    experiment_name="xgb_quantile_test",
    evaluate_on_test=True
)

Training models: 100%|██████████| 3/3 [00:01<00:00,  2.13it/s]


In [20]:
sample_result_1 = results_lightning_std.training_results[0]
print(f"Model type: {sample_result_1.model_type}")
print(f"Strategy: {sample_result_1.modeling_strategy.value}")
print(f"SKU tuples: {sample_result_1.sku_tuples}")
print(f"Quantile level: {sample_result_1.quantile_level}")
print(f"quantile_score: {sample_result_1.performance_metrics.get('quantile_score', 'N/A')}")
print(f"predictions: {sample_result_1.performance_metrics.get('predictions', 'N/A')}")
print("overall stats: ----------------------------------------------------")
print(f"number of models trained: {len(results_lightning_std.training_results)}")

Model type: xgboost_quantile
Strategy: individual
SKU tuples: [(1912, 7)]
Quantile level: 0.5
quantile_score: N/A
predictions: [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
overall stats: ----------------------------------------------------
number of models trained: 15


In [21]:
sample_result_1.performance_metrics

{'quantile_losses': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. , 0. ,
        0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. ,
        0. , 0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0. , 0. ,
        0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ,
        0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]),
 'predictions': array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0

In [9]:
release_manager = ReleaseManager()
output_dir = Path("./xgb_releases_2")
release_path = release_manager.create_complete_release(
    experiment_results=results_lightning_std,  # Your ExperimentResults from pipeline
    base_output_dir=output_dir
  )

# Check Data in pipeline and prototype for consistency

In [14]:
sku = (715, 7)
loader = DataLoader(data_config)
loader.load_data(lazy=False)
dataset = loader.prepare_modeling_dataset([sku], ModelingStrategy.INDIVIDUAL)

In [15]:
loader = DataLoader(data_config)
# Don't call load_data() at all - let prepare_modeling_dataset do it
dataset = loader.prepare_modeling_dataset([sku], ModelingStrategy.INDIVIDUAL)

In [72]:
# Access data
X_train_pipeline = dataset.X_train
y_train_pipeline = dataset.y_train
X_test_pipeline = dataset.X_test
y_test_pipeline = dataset.y_test

In [22]:
X_train_pipeline.shape

(741, 132)

In [34]:

# Data loading
mapping_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/feature_mapping_train.pkl"
features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather"

with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

train_features = pl.read_ipc(features_path)
train_target = pl.read_ipc(target_path)

In [35]:
X = (train_features
     .lazy()
     .filter(pl.col('productID') == 715)
     .filter(pl.col('storeID') == 7)
     .drop_nulls()
     .sort("date","skuID")
     .collect())

X_bdIDs = (X
           .lazy()
           .select('bdID')
           .unique()
           .collect()
           .to_numpy()
           .flatten())

y = (train_target
     .lazy()
     .filter(pl.col("bdID").is_in(X_bdIDs))
     .join(
         X.lazy().select("bdID","skuID"), 
         on="bdID", 
         how="left")
     .sort("date","skuID")
     .collect())

In [36]:
split_date = pl.date(2016, 1, 1)
X_train = X.filter(pl.col("date") < split_date)
X_test = X.filter(pl.col("date") >= split_date)
y_train = y.filter(pl.col("date") < split_date)
y_test = y.filter(pl.col("date") >= split_date)

In [37]:
meta_cols = ['frequency',
 'idx',
 'bdID',
 'base_date',
 'date',
 'dateID',
 'skuID',
 'productID',
 'storeID',
 'companyID',
 'is_daily',
 'missing_value',
 'not_for_sale',
 'name',
 'name-2']

In [38]:
# Explicitly set dtype to float64 for all arrays
X_train = (X_train
            .sort("date","skuID")
            .select(pl.selectors.exclude(meta_cols)))

y_train = (y_train
            .sort("date","skuID")
            .select("target")
            .to_numpy())  # Also flatten to ensure 1D

X_test = (X_test
        .sort("date","skuID")
        .select(pl.selectors.exclude(meta_cols)))

y_test = (y_test
        .sort("date","skuID")
        .select("target"))

In [ ]:
#wrap them into print statements to see the output
print("Train set shape test:   ", X_train.shape == X_train_pipeline.shape )
print("Train label shape test: ", y_train.shape == y_train_pipeline.shape )
print(30*"-")
print("Test set shape test:  ", X_test.shape == X_test_pipeline.shape )
print("Test label shape test:", y_test.shape == y_test_pipeline.shape ) 

Train set shape test:    True
Train label shape test:  True
------------------------------
Test set shape test:   True
Test label shape test: True


In [ ]:
#test if prototype and pipeline data are the same
print("Training set check:    ", (X_train == X_train_pipeline).to_numpy().all())
print("Training label check:  ",(y_train == y_train_pipeline).all())
print(30*"-")
print("Test set check:   ", (X_test == X_test_pipeline).to_numpy().all())
print("Test label check: ",(y_test == y_test).to_numpy().all())  

Training set check:     True
Training label check:   True
------------------------------
Test set check:    True
Test label check:  True


# Test Hyperparameter mode

## Test Tune Run

In [10]:
data_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather"
df_clean = pl.read_ipc(data_path)

sku_tuples = [(d['productID'], d['storeID']) for d in df_clean.select(pl.col("productID"), pl.col("storeID")).unique().to_dicts()]
len(sku_tuples)

7634

In [11]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/processed/train_data_features.feather",
    target_path = "/Users/ivn/Documents/PhD/Transformer Research/Code/Benchmarking/data/db_snapshot_offsite/train_data/train_data/train_data_target.feather",
    split_date="2016-01-01",
)

pipeline = BenchmarkPipeline(data_config)

In [ ]:

# Step 1: Tune hyperparameters
tune_result = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.COMBINED,
    model_type="xgboost_quantile",
    quantile_alphas=[0.5],
    mode="hp_tune",
    tune_on=500,
    tuning_config={'n_trials': 100, 'n_folds': 5}
)

[I 2025-10-12 11:35:00,938] A new study created in memory with name: no-name-7003e709-a98f-40b2-86fb-b8ba2eb03329
Best trial: 0. Best value: 0.452842:   1%|          | 1/100 [01:24<2:19:57, 84.83s/it]

[I 2025-10-12 11:36:25,771] Trial 0 finished with value: 0.4528419431700338 and parameters: {'eta': 0.11861663446573512, 'max_depth': 10, 'min_child_weight': 22, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'gamma': 0.7799726016810132, 'reg_alpha': 0.5808361216819946, 'reg_lambda': 8.675143843171858, 'n_estimators': 200}. Best is trial 0 with value: 0.4528419431700338.


Best trial: 0. Best value: 0.452842:   2%|▏         | 2/100 [02:10<1:41:06, 61.91s/it]

[I 2025-10-12 11:37:11,632] Trial 1 finished with value: 0.4616433346930238 and parameters: {'eta': 0.21534104756085318, 'max_depth': 3, 'min_child_weight': 30, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'gamma': 0.9091248360355031, 'reg_alpha': 1.8340450985343382, 'reg_lambda': 3.1119982052994235, 'n_estimators': 181}. Best is trial 0 with value: 0.4528419431700338.


Best trial: 0. Best value: 0.452842:   3%|▎         | 3/100 [02:46<1:20:27, 49.76s/it]

...

[I 2025-10-12 14:07:30,735] Trial 97 finished with value: 0.43691003917457005 and parameters: {'eta': 0.24841264826634638, 'max_depth': 10, 'min_child_weight': 29, 'subsample': 0.9900795239964222, 'colsample_bytree': 0.6791682272044169, 'gamma': 1.0502498149815178, 'reg_alpha': 7.746339740715483, 'reg_lambda': 4.952198672774244, 'n_estimators': 288}. Best is trial 92 with value: 0.4355871870782814.


Best trial: 92. Best value: 0.435587:  99%|█████████▉| 99/100 [2:34:06<01:48, 108.32s/it]

[I 2025-10-12 14:09:07,053] Trial 98 finished with value: 0.43784018466134433 and parameters: {'eta': 0.23918779886546454, 'max_depth': 10, 'min_child_weight': 26, 'subsample': 0.9708716432023022, 'colsample_bytree': 0.6577514026933193, 'gamma': 1.178210651753037, 'reg_alpha': 7.082043026087628, 'reg_lambda': 3.875218584516394, 'n_estimators': 262}. Best is trial 92 with value: 0.4355871870782814.


Best trial: 92. Best value: 0.435587: 100%|██████████| 100/100 [2:35:58<00:00, 93.59s/it] 

[I 2025-10-12 14:10:59,791] Trial 99 finished with value: 0.43689128808433964 and parameters: {'eta': 0.2148414877349589, 'max_depth': 10, 'min_child_weight': 28, 'subsample': 0.9550520499448616, 'colsample_bytree': 0.6912324156362424, 'gamma': 1.4319926253223472, 'reg_alpha': 6.733980593780706, 'reg_lambda': 4.426250717424357, 'n_estimators': 282}. Best is trial 92 with value: 0.4355871870782814.


In [15]:
tune_result.best_score

0.4355871870782814

In [16]:
tune_result.best_params

{'eta': 0.2251586017238223,
 'max_depth': 10,
 'min_child_weight': 26,
 'subsample': 0.9639446007829685,
 'colsample_bytree': 0.6507755877543546,
 'gamma': 0.6139963456942783,
 'reg_alpha': 6.768595449859619,
 'reg_lambda': 4.832153974114423,
 'n_estimators': 293}

{'eta': 0.2251586017238223,
 'max_depth': 10,
 'min_child_weight': 26,
 'subsample': 0.9639446007829685,
 'colsample_bytree': 0.6507755877543546,
 'gamma': 0.6139963456942783,
 'reg_alpha': 6.768595449859619,
 'reg_lambda': 4.832153974114423,
 'n_estimators': 293}

## Run With tuned Params

In [27]:
sku_tuples=[(1912, 7), (377, 1), (715, 7)]
quantiles = [0.5, 0.7, 0.9, 0.95, 0.99]

hyperparameters = {'eta': 0.2251586017238223,
 'max_depth': 10,
 'min_child_weight': 26,
 'subsample': 0.9639446007829685,
 'colsample_bytree': 0.6507755877543546,
 'gamma': 0.6139963456942783,
 'reg_alpha': 6.768595449859619,
 'reg_lambda': 4.832153974114423,
 'n_estimators': 293}

pipeline = BenchmarkPipeline(data_config)

In [63]:
results_lightning_std = pipeline.run_experiment(
    sku_tuples=sku_tuples_complete[:500],
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="xgboost_quantile",
    quantile_alphas=quantiles,
    hyperparameters = hyperparameters,
    experiment_name="xgb_quantile_test",
    evaluate_on_test=True
)

Training models: 100%|██████████| 500/500 [09:05<00:00,  1.09s/it]


In [64]:
print(f"number of models trained: {len(results_lightning_std.training_results)}")
print(f"number of skus: {len(sku_tuples_complete[:100])}")
print(f"number of quantiles: {len(quantiles)}")

number of models trained: 2500
number of skus: 100
number of quantiles: 5


In [65]:
results_lightning_std.training_results[1].performance_metrics.get('mean_quantile_loss')

np.float64(0.23566433566433567)

In [66]:
# More comprehensive DataFrame with additional metrics
results_df = pl.DataFrame({
    "sku_tuples": [result.sku_tuples for result in results_lightning_std.training_results],
    "productID": [result.sku_tuples[0][0] for result in results_lightning_std.training_results],
    "storeID": [result.sku_tuples[0][1] for result in results_lightning_std.training_results],
    "quantile_level": [result.quantile_level for result in results_lightning_std.training_results],
    "mean_quantile_loss" : [result.performance_metrics.get('mean_quantile_loss') for result in results_lightning_std.training_results]
})

results_df

sku_tuples,productID,storeID,quantile_level,mean_quantile_loss
list[list[i64]],i64,i64,f64,f64
"[[251, 1]]",251,1,0.5,0.244755
"[[251, 1]]",251,1,0.7,0.235664
"[[251, 1]]",251,1,0.9,0.165035
"[[251, 1]]",251,1,0.95,0.107343
"[[251, 1]]",251,1,0.99,0.037972
…,…,…,…,…
"[[2266, 7]]",2266,7,0.5,0.066434
"[[2266, 7]]",2266,7,0.7,0.091608
"[[2266, 7]]",2266,7,0.9,0.107692


In [72]:
results_df.sort('mean_quantile_loss', descending=True).head(10)

sku_tuples,productID,storeID,quantile_level,mean_quantile_loss
list[list[i64]],i64,i64,f64,f64
"[[2284, 9]]",2284,9,0.7,5.529371
"[[466, 7]]",466,7,0.5,5.472028
"[[2284, 9]]",2284,9,0.5,5.020979
"[[2284, 9]]",2284,9,0.9,4.745455
"[[466, 7]]",466,7,0.7,4.541259
"[[2284, 9]]",2284,9,0.95,4.512587
"[[46, 3]]",46,3,0.5,4.48951
"[[46, 3]]",46,3,0.7,4.224476
"[[2284, 9]]",2284,9,0.99,4.130559


In [73]:
(df_clean
 .filter(pl.col('productID') == 2284)
 .filter(pl.col('storeID') == 9)
 .filter(pl.col('date') < pl.date(2016,1,1))
 .drop_nulls()
 .select(pl.col('target_lag_1').mean()))

target_lag_1
f64
1.316583


In [77]:
(df_clean
 .filter(pl.col('productID') == 2284)
 .filter(pl.col('storeID') == 9)
 .filter(pl.col('date') >= pl.date(2016,1,1))
 .drop_nulls()
 .select(pl.col('target_lag_1').mean()))

target_lag_1
f64
21.195804


In [50]:
from lets_plot import *

LetsPlot.setup_html()

# Convert quantile_level to string for proper categorical handling
results_df_plot = results_df.with_columns(
    pl.col('quantile_level').cast(pl.Utf8).alias('quantile_level')
)

ggplot(results_df_plot, aes(x='quantile_level', y='mean_quantile_loss')) + \
    geom_boxplot(fill='#69b3a2', alpha=0.5, outlier_shape=21) + \
    geom_jitter(width=0.1, alpha=0.3, size=2, color='#404080') + \
    labs(
        title='Mean Quantile Loss Distribution by Quantile Level',
        subtitle=f'Based on {len(results_df)} model results',
        x='Quantile Level',
        y='Mean Quantile Loss'
    ) + \
    theme_minimal() + \
    theme(plot_title=element_text(size=14, face='bold'))

In [49]:
ggplot(results_df, aes(x='quantile_level', y='mean_quantile_loss')) + \
    geom_boxplot(fill='#69b3a2', alpha=0.5, outlier_shape=21) + \
    geom_jitter(width=0.1, alpha=0.3, size=2, color='#404080') + \
    labs(
        title='Mean Quantile Loss Distribution by Quantile Level',
        subtitle=f'Based on {len(results_df)} model results',
        x='Quantile Level',
        y='Mean Quantile Loss'
    ) + \
    theme_minimal() + \
    theme(plot_title=element_text(size=14, face='bold'))